In [6]:
from selenium import webdriver
import time
from calendar import monthrange
import calendar
from datetime import datetime

In [87]:
def get_hours(iso, peak_type, period):
    
    url = 'https://www.energygps.com/HomeTools/PowerCalendar'

    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome('./chromedriver.exe', options=chrome_options) 
    driver.get(url)
    time.sleep(2) 

    input_startDate = driver.find_element_by_id('startDate')
    input_endDate = driver.find_element_by_id('endDate')
    
    year = int(period[:4])
    mon2id = {month: index for index, month in enumerate(calendar.month_abbr) if month}
    if '-' in period:
        f = 4
        month = period.split('-')[1]
        day = monthrange(year, period.split('-')[1])[1]
        start = datetime(year, month, 1)
        end = datetime(year, month, monthrange(year, period.split('-')[1])[1])
    elif 'Q' in period:
        f = 10
        q = int(period[-1])
        start = datetime(year, 3*q-2, 1)
        end = datetime(year, 3*q, monthrange(year, 3*q)[1])
    elif 'A' in period:
        f = 26
        start = datetime(year, 1, 1)
        end = datetime(year, 12, 1)
    else:
        f = 4
        m = mon2id[period[4:]]
        start = datetime(year, m, 1)
        end = datetime(year, m, monthrange(year, m)[1])
    
    input_startDate.clear()
    input_startDate.send_keys(start.strftime('%m/%d/%Y'))
    input_endDate.clear()
    input_endDate.send_keys(end.strftime('%m/%d/%Y'))
    
    select_region = driver.find_element_by_xpath("//*[@id='isoregion']/option[text()='" + iso + "']")
    select_region.click()

    view_button = driver.find_element_by_xpath('//*[@id="main"]/form/table/tbody/tr[2]/td[4]/input[1]')
    view_button.click()

    if peak_type == 'onpeak':
        ith = 4
    elif peak_type == 'offpeak':
        ith = 5
    else:
        ith = 3
    if 'A' in period:
        ith -= 1
    hours = driver.find_element_by_xpath('//*[@id="main"]/table[2]/tbody/tr/td[1]/table/tbody/tr[' + str(f) + ']/td[' + str(ith) + ']').text
    return(hours, peak_type, start.strftime('%m/%d/%Y'), end.strftime('%m/%d/%Y'))

In [88]:
get_hours('ERCOT', 'onpeak', '2019May')

('352', 'onpeak', '05/01/2019', '05/31/2019')